In [6]:
import pandas as pd
import numpy as np
departments = pd.read_csv('.\\data\\departments.csv', sep=',', encoding="ISO-8859-1")
products = pd.read_csv('.\\data\\products.csv', sep=',', encoding="ISO-8859-1")
mergedOrders = pd.read_csv('.\\data\\mergedOrders.csv', sep=',', encoding="ISO-8859-1")
orders1 = pd.read_csv('.\\data\\orders.csv', sep=',', encoding="ISO-8859-1")

In [164]:
products.head()
departments.head()
mergedOrders.head()

,Unnamed: 0,order_id,product_id,reordered,user_id,days_since_prior_order
0,0,8,23423,1,3107,17.0
1,1,40,10070,1,382,23.0
2,2,40,42450,1,382,23.0
3,3,40,33198,1,382,23.0
4,4,40,34866,1,382,23.0


In [253]:
#mergedOrders = mergedOrders.drop(columns = ["order_id"])
dfDaysAverage = mergedOrders.groupby("product_id").mean()
dfDaysAverage.sort_values("days_since_prior_order").dropna()
dfDaysAverage = dfDaysAverage.drop(columns=["Unnamed: 0","order_id","reordered","user_id"])
dfDaysAverage = dfDaysAverage.rename(columns={"days_since_prior_order": "mediaEcompras"})
dfDaysAverage = dfDaysAverage[dfDaysAverage["mediaEcompras"] > 0]
dfDaysAverage['product_id'] = dfDaysAverage.index
dfDaysAverage.head()

,mediaEcompras,product_id
product_id,,
1,12.333333,1
2,20.500000,2
3,19.250000,3
4,12.000000,4
7,10.666667,7


In [254]:
dfQuantity = mergedOrders.groupby("product_id").count()
dfQuantity = dfQuantity.drop(columns=["Unnamed: 0","order_id","reordered","days_since_prior_order"]).reset_index()
dfQuantity = dfQuantity.rename(columns={"user_id": "cantitatMitjaProductes"})
dfQuantity = dfQuantity.sort_values('cantitatMitjaProductes')
dfQuantity['product_id'] = dfQuantity.index
dfQuantity.head()

,product_id,cantitatMitjaProductes
20144,20144,1
28037,28037,1
13241,13241,1
13242,13242,1
13243,13243,1


In [255]:
allMergedOrders = mergedOrders
aux = allMergedOrders.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
allMergedOrders = allMergedOrders.groupby( [ "product_id", "user_id"] ).mean().reset_index()
allMergedOrders = allMergedOrders.drop(columns=["Unnamed: 0","order_id","reordered"]).reset_index()
allMergedOrders = pd.merge(aux,allMergedOrders, on=[ "user_id", "product_id"])
allMergedOrders = allMergedOrders.rename(columns={"days_since_prior_order": "media_usuarioEcompras"})
allMergedOrders = allMergedOrders.drop(columns=["index"])
allMergedOrders.head()

,user_id,product_id,total,media_usuarioEcompras
0,1,196,11,19.000000
1,1,10258,10,19.000000
2,1,10326,1,28.000000
3,1,12427,10,19.555556
4,1,13032,4,19.750000


In [262]:
dfProva = allMergedOrders
#dfProva = dfProva.groupby(["user_id",'product_id']).agg({'product_id':['mean', 'median']})
#dfProva['puntuacio'] = dfProva.age * dfProva.bmi
#dfProva = dfProva.groupby( [ "user_id", "product_id"] ).size().to_frame(name = 'total').reset_index()
dfProva =  pd.merge(dfProva, dfQuantity, on="product_id")
dfProva =  pd.merge(dfProva, dfDaysAverage, on="product_id")
dfProva['puntCant'] = np.where(dfProva.total > dfProva.cantitatMitjaProductes, 5, dfProva.total-dfProva.cantitatMitjaProductes+5)
dfProva['punTemps'] = np.where(dfProva.media_usuarioEcompras > dfProva.mediaEcompras, 0, 5)
dfProva['punTotal'] = np.where(dfProva.media_usuarioEcompras > dfProva.mediaEcompras, dfProva.puntCant+, 5)
dfProva

C:\Users\Joan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: FutureWarning: 'product_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_id,product_id,total,media_usuarioEcompras,cantitatMitjaProductes,mediaEcompras,puntCant,punTemps
0,1,196,11,19.000000,9,12.160839,5,0
1,15,196,5,8.600000,9,12.160839,1,5
2,19,196,3,6.666667,9,12.160839,-1,5
3,21,196,1,6.000000,9,12.160839,-3,5
4,31,196,2,7.500000,9,12.160839,-2,5
5,43,196,2,3.500000,9,12.160839,-2,5
6,52,196,14,9.538462,9,12.160839,5,5
7,67,196,20,7.526316,9,12.160839,5,5
8,81,196,2,22.000000,9,12.160839,-2,0
9,82,196,9,11.500000,9,12.160839,5,5


In [263]:

dfTotal = dfQuantity
dfTotal = pd.merge(dfDaysAverage, dfTotal, on="product_id")

C:\Users\Joan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: FutureWarning: 'product_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [264]:
#dfTotal = dfTotal.drop(dfTotal[dfTotal.days_since_prior_order == 1].index)

In [265]:
dfTotal = dfTotal.sort_values("cantitat_productes")
dfTotal

KeyError: 'cantitat_productes'

In [266]:
dfSmaller = dfTotal.iloc[: 16305]
productsSmaller = list(dfSmaller.index)
dfSmaller.sort_values("media_entre_compras")

KeyError: 'media_entre_compras'

In [267]:
dfSmall = dfTotal.iloc[16305: 24364:]
productsSmall = list(dfSmall.index)
dfSmall.sort_values("days_since_prior_order")

KeyError: 'days_since_prior_order'

In [268]:
dfMedium = dfTotal.iloc[24364: 29543:]
productsMedium = list(dfMedium.index)
dfMedium.sort_values("days_since_prior_order")

KeyError: 'days_since_prior_order'

In [269]:
dfBig = dfTotal.iloc[29543: 31015]
productsBig = list(dfBig.index)
dfBig.sort_values("days_since_prior_order")

KeyError: 'days_since_prior_order'

In [270]:
dfBigger = dfTotal.iloc[31015:]
productsBigger = list(dfBigger.index)
dfBigger.sort_values("days_since_prior_order")

KeyError: 'days_since_prior_order'

In [ ]:
dfBigger.sort_values("days_since_prior_order")